In [2]:
!pip install topmost

In [3]:
import topmost
import torch
from torch import nn
import torch.nn.functional as F
from topmost.models.dynamic.DETM import DETM
from topmost import eva
import numpy as np
import itertools
import json
import joblib
from tqdm import tqdm
import os
import json
import random

In [4]:
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from topmost.data.file_utils import split_text_word

## Funciones externas

In [6]:
def get_theta_redefined(self, bows, times, eta=None): ## amortized inference
        """Returns the topic proportions.
        """
        # normalized_bows = bows / bows.sum(1, keepdims=True)
        # print("🍀 Before", normalized_bows)

        normalized_bows = bows / (bows.sum(1, keepdims=True) + 1e-12)
        # print("🍀 After", normalized_bows)
    
        if eta is None and self.training is False:
            eta, kl_eta = self.get_eta(self.rnn_inp)

        eta_td = eta[times]
        inp = torch.cat([normalized_bows, eta_td], dim=1)
        q_theta = self.q_theta(inp)
        if self.enc_drop > 0:
            q_theta = self.t_drop(q_theta)
        mu_theta = self.mu_q_theta(q_theta)
        logsigma_theta = self.logsigma_q_theta(q_theta)
        z = self.reparameterize(mu_theta, logsigma_theta)
        theta = F.softmax(z, dim=-1)
        kl_theta = self.get_kl(mu_theta, logsigma_theta, eta_td, torch.zeros(self.num_topics).to(self.device))

        if self.training:
            return theta, kl_theta
        else:
            return theta

DETM.get_theta = get_theta_redefined

In [7]:
class DETMFixed(DETM):
    def __init__(self, vocab_size, num_times, train_size, train_time_wordfreq, num_topics=50, train_WE=True, pretrained_WE=None, en_units=800, eta_hidden_size=200, rho_size=300, enc_drop=0.0, eta_nlayers=3, eta_dropout=0.0, delta=0.005, theta_act='relu', device='cpu'):
        super().__init__(vocab_size, num_times, train_size, train_time_wordfreq, num_topics, True, pretrained_WE, en_units, eta_hidden_size, rho_size, enc_drop, eta_nlayers, eta_dropout, delta, theta_act, device)
        
        ## define the word embedding matrix \rho
        if not self.train_WE:
            rho = nn.Embedding(pretrained_WE.shape[1], pretrained_WE.shape[0])
            rho.weight.data = torch.from_numpy(pretrained_WE)
            self.rho = rho.weight.data.clone().float().to(self.device)

In [8]:
def _coherence_modified(reference_corpus, vocab, top_words, cv_type='c_v'):
    # print('🍀')
    
    split_top_words = split_text_word(top_words)
    num_top_words = len(split_top_words[0])
    for item in split_top_words:
        assert num_top_words == len(item)

    split_reference_corpus = split_text_word(reference_corpus)
    dictionary = Dictionary(split_text_word(vocab))

    cm = CoherenceModel(texts=split_reference_corpus, dictionary=dictionary, topics=split_top_words, topn=num_top_words, coherence=cv_type)
    cv_per_topic = cm.get_coherence_per_topic()
    # print(f"Coherence scores per topic: {cv_per_topic}")

    valid_scores = [score for score in cv_per_topic if not np.isnan(score)]
    if not valid_scores:
        raise ValueError("All coherence scores are NaN.")
    score = np.mean(valid_scores)

    return score

In [9]:
eva.topic_coherence._coherence = _coherence_modified

def eval(trainer, top_words):
    # get theta (doc-topic distributions)
    train_theta, test_theta = trainer.export_theta()

    train_times = dataset.train_times.cpu().numpy()
    # compute topic coherence
    dynamic_TC = eva.dynamic_coherence(dataset.train_texts, train_times, dataset.vocab, top_words, verbose=True)
    print("dynamic_TC: ", dynamic_TC)

    # compute topic diversity
    dynamic_TD = eva.dynamic_diversity(top_words, dataset.train_bow.cpu().numpy(), train_times, dataset.vocab, verbose=False)
    print("dynamic_TD: ", dynamic_TD)

    # evaluate clustering
    clustering = eva._clustering(test_theta, dataset.test_labels)
    print(clustering)

    # evaluate classification
    classification = eva._cls(train_theta, test_theta, dataset.train_labels, dataset.test_labels)
    print(classification)
    
    json = {
        "dynamic_TC": dynamic_TC,
        "dynamic_TD": dynamic_TD,
        "clustering": clustering,
        "classification": classification
    }
    
    return json, dynamic_TC, dynamic_TD, clustering, classification

In [10]:
import joblib
import json

def save_txt(obj, path):
    with open(path, 'w') as file:
        file.write(obj)

def save_top_words_txt(top_words, path):
    with open(os.path.join(path, 'top_words.txt'), 'w') as file:
        for i, time in enumerate(top_words):
            file.write(f'================= Time {i} ================= \n')
            for j, topic in enumerate(time):
                file.write(f'================= Topic {j} ================= \n')
                file.write(f'{topic}')
                file.write('\n')
            file.write('\n')

def save_json(obj, path):
    with open(path, 'w') as file:
        json.dump(obj, file)

def save_joblib(obj, path):
    joblib.dump(obj, path)

def get_desc(num_topics, rho_size, en_units, eta_hidden_size, enc_drop, eta_nlayers, eta_dropout, delta, theta_act, learning_rate, epochs):
    return f'Trainig model with hyperparameters: \n num_topics (K):  {num_topics} rho_size: {rho_size} \n en_units: {en_units} \n eta_hidden_size: {eta_hidden_size} \n enc_drop: {eta_dropout} \n eta_nlayers: {eta_nlayers} \n eta_dropou: {eta_dropout} \n delta: {delta} \n theta_act: {theta_act} \n learning_rate: {learning_rate} \n epochs: {epochs}'

def get_hyperparameters_json(config):
    return { 
        "num_topics" : config['num_topics'],
        "rho_size": config['rho_size'],
        "en_units" : config['en_units'],
        "eta_hidden_size" : config['eta_hidden_size'],
        "enc_drop": config['enc_drop'],
        "eta_nlayers" : config['eta_nlayers'],
        "eta_dropou" : config['eta_dropout'],
        "delta" : config['delta'],
        "theta_act" : config['theta_act'],
        "learning_rate": config['learning_rate'],
        "epochs": config['epochs']}
    
# def get_hyperparameter_space():
    # with open('./hyperparameter_space.json', 'r') as f:
        # return json.load(f)

def get_hyperparameter_space():
    #with open("hyperparameter_space.json", 'r') as f:
        #return json.load(f)
    return {
        "num_topics": [20, 50, 100],
        "rho_size": [200, 300, 400],
        "en_units": [400, 800, 1200],
        "eta_nlayers": [2, 3, 4],
        "delta": [0.001, 0.005, 0.01, 0.05],
        "theta_act": ['anh', 'softplus', 'relu', 'rrelu', 'leakyrelu', 'elu', 'selu', 'glu'],
        "eta_hidden_size": [100, 200, 300],
    
        # Parametros del optimizador
        
        "enc_drop": [0.0, 0.2, 0.3, 0.5],
        "eta_dropout": [0.0, 0.2, 0.3, 0.5],
        "learning_rate": [1e-3, 5e-4, 0.02],
        "epochs": [200, 400, 800]
    }

def get_default_config():
    return { 
        "num_topics" : 50,
        "rho_size": 300,
        "en_units" : 800,
        "eta_hidden_size" : 200,
        "enc_drop": 0.0,
        "eta_nlayers" : 3,
        "eta_dropout" : 0.0,
        "delta" : 0.005,
        "theta_act" : 'relu',
        "learning_rate": 0.005,
        "epochs": 100}

def random_configuration(hyperparameter_space):
    return {
        'num_topics': random.choice(hyperparameter_space["num_topics"]),
        'rho_size': random.choice(hyperparameter_space["rho_size"]),
        'en_units': random.choice(hyperparameter_space["en_units"]),
        'eta_hidden_size': random.choice(hyperparameter_space["eta_hidden_size"]),
        'enc_drop': random.choice(hyperparameter_space["enc_drop"]),
        'eta_nlayers': random.choice(hyperparameter_space["eta_nlayers"]),
        'eta_dropout': random.choice(hyperparameter_space["eta_dropout"]),
        'delta': random.choice(hyperparameter_space["delta"]),
        'theta_act': random.choice(hyperparameter_space["theta_act"]),
        'learning_rate': random.choice(hyperparameter_space["learning_rate"]),
        'epochs': random.choice(hyperparameter_space["epochs"])}

### --------------------------------------------------------

In [11]:
def save_result(top_words, trainer, config, metrics_json):
    global samples 
    path = os.path.join(output_dir, f'{samples}')
    os.makedirs(path, exist_ok=True)

    try:
        save_top_words_txt(top_words, path)
        save_json(config, os.path.join(path, 'hyperparameters.json'))
        save_json(metrics_json, os.path.join(path, 'metrics_json.json'))
        save_json({"config": config, "metrics": metrics_json}, os.path.join(path, 'metrics_config.json'))
        #save_joblib(trainer, os.path.join(path, 'trainer.joblib'))
        save_joblib(top_words, os.path.join(path, 'top_words.joblib'))
    except Exception as e:
        traceback.print_exc()

In [35]:
import traceback

def evaluate_configuration(config):
    global samples, tested_configs, output_dir
    
    config_key = tuple(config.values())
                       
    if config_key in tested_configs.keys():
        return tested_configs[config_key]
    
    model = DETMFixed(
            num_topics=config['num_topics'], 
            vocab_size=dataset.vocab_size,
            num_times=dataset.num_times,
            train_size=dataset.train_size,
            train_time_wordfreq=dataset.train_time_wordfreq,
            train_WE=False, 
            pretrained_WE=dataset.pretrained_WE, 
            en_units=config['en_units'], 
            eta_hidden_size=config['eta_hidden_size'], 
            rho_size=config['rho_size'], 
            enc_drop=config['enc_drop'], 
            eta_nlayers=config['eta_nlayers'], 
            eta_dropout=config['eta_dropout'], 
            delta=config['delta'], 
            theta_act=config['theta_act'],
            device=device)

    model = model.to(device)
    trainer = topmost.DynamicTrainer(model, dataset, epochs=config['epochs'], learning_rate=config['learning_rate'], verbose=False)
    top_words, train_theta = trainer.train()
    
    try:
        metrics_json, dynamic_TC, dynamic_TD, clustering, classification = eval(trainer, top_words)
        tested_configs[config_key] = {"dynamic_TC": dynamic_TC, "dynamic_TD": dynamic_TD}
        save_result(top_words, trainer, config, metrics_json)
        samples += 1
    except Exception as e:
        tested_configs[config_key] = {"dynamic_TC": float('-inf'), "dynamic_TD": float('-inf')}
        print(e)
        
    save_joblib(tested_configs, os.path.join(output_dir, 'tested_configs.joblib'))
    
    return tested_configs[config_key]

In [39]:
def key_func(value):
    try:
        return 0.5 * value[1]["dynamic_TC"] + 0.5 * value[1]["dynamic_TD"]
    except: 
        return value[1]

def evolutionary_search(search_space, generations=10, mutation_rate=0.1, population_size=20, population=None, generation_count=0):
    global samples
    
    if not population:
        population = [random_configuration(search_space) for _ in range(population_size - 1)]
        population.append(get_default_config())

    # print(population)
    
    for generation in range(generation_count, generations):
        scores = [(config, evaluate_configuration(config)) for config in population]
        scores.sort(key=key_func, reverse=True)  

        print(f"🧬 Generación {generation + 1}, mejor resultado: {scores[0][1]}")
        
        num_parents = population_size // 2
        parents = [config for config, _ in scores[:num_parents]]
        
        children = []
        while len(children) < population_size - num_parents:
            parent_1, parent_2 = random.sample(parents, 2)
            child = {key: random.choice([parent_1[key], parent_2[key]]) for key in search_space.keys()}
            
            if random.random() < mutation_rate:
                param_to_mutate = random.choice(list(search_space.keys()))
                print(param_to_mutate)
                child[param_to_mutate] = random.choice(search_space[param_to_mutate])
            
            children.append(child)
        
        population = parents + children
        joblib.dump(population, os.path.join(output_dir, 'population.joblib'))
    
    best_config, best_score = max(scores, key=key_func)
    return best_config, best_score

## Main

In [14]:
device = "cuda"  # or "cpu"
dataset_dir = "/kaggle/input/revista-de-ciencias-mdicas-de-la-habana-cuba"
output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)
dataset = topmost.data.DynamicDataset(dataset_dir, batch_size=200, read_labels=True, device=device)

train size:  1208
test size:  658
vocab size:  19384
average length: 1335.378
num of each time slice:  22 [20 34 46 41 56 20 76 58 43 54 83 71 64 67 63 60 55 83 75 58 57 24]


In [15]:
output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)

In [16]:
import os
import json

generation = 0

results_path = [dir_.path for dir_ in os.scandir(output_dir) if not dir_.is_file()]
samples = len(results_path)

print(len(results_path))

88


In [26]:
import joblib
population_size = 50
# population = None
# population = joblib.load('/kaggle/input/datos-de-inicio-del-entrenamiento/population.joblib')
population = joblib.load('/kaggle/working/results/population.joblib')

In [33]:
import joblib
# tested_configs = dict()
# tested_configs = joblib.load('/kaggle/input/datos-de-inicio-del-entrenamiento/tested_configs.joblib')
tested_configs = joblib.load('/kaggle/working/results/tested_configs.joblib')
# print(tested_configs)

In [40]:
hyperparameter_space = get_hyperparameter_space()

best_config, best_score = evolutionary_search(hyperparameter_space, population_size=population_size, generations=100, mutation_rate=0.2, population=population, generation_count=generation)
print(best_config)
print(best_score)

🧬 Generación 1, mejor resultado: {'dynamic_TC': 0.4475481219000681, 'dynamic_TD': 0.6240909090909091}
rho_size
eta_nlayers
theta_act
epochs
rho_size
🧬 Generación 2, mejor resultado: {'dynamic_TC': 0.4475481219000681, 'dynamic_TD': 0.6240909090909091}
delta
enc_drop
num_topics
learning_rate
learning_rate
theta_act
🧬 Generación 3, mejor resultado: {'dynamic_TC': 0.4475481219000681, 'dynamic_TD': 0.6240909090909091}
en_units
eta_dropout
enc_drop
🧬 Generación 4, mejor resultado: {'dynamic_TC': 0.4475481219000681, 'dynamic_TD': 0.6240909090909091}
enc_drop
theta_act
🧬 Generación 5, mejor resultado: {'dynamic_TC': 0.4475481219000681, 'dynamic_TD': 0.6240909090909091}
learning_rate
enc_drop
num_topics
eta_dropout
eta_nlayers
en_units
rho_size
num_topics
🧬 Generación 6, mejor resultado: {'dynamic_TC': 0.4475481219000681, 'dynamic_TD': 0.6240909090909091}
rho_size
num_topics
theta_act
learning_rate
learning_rate
🧬 Generación 7, mejor resultado: {'dynamic_TC': 0.4475481219000681, 'dynamic_TD': 0

## Evaluando resultados

In [ ]:
import os

output_dir = '/kaggle/working/results'
results_path = [dir_.path for dir_ in os.scandir(output_dir) if not dir_.is_file()]

print(results_path)

In [ ]:
old_tc = float('-inf')
TC = []
old_td = float('-inf')
TD = []
old_tc_td = float('-inf')
TC_TD = []

for path in results_path:
    with open(os.path.join(path, 'metrics_json.json'), 'r') as file:
        metrics = json.load(file)

    cr_tc = metrics["dynamic_TC"]
    if cr_tc >= old_tc:
        with open(os.path.join(path, 'metrics_config.json'), 'r') as file:
            new = json.load(file)
        if cr_tc == old_tc:
            TC.append((new, path))
        else:
            old_tc = cr_tc
            TC = [(new, path)]

    cr_td = metrics["dynamic_TD"]
    if cr_td >= old_td:
        with open(os.path.join(path, 'metrics_config.json'), 'r') as file:
            new = json.load(file)
        if cr_td == old_td:
            TD.append((new, path))
        else:
            old_td = cr_td
            TD = [(new, path)]

    cr_tc_td = 0.5 * cr_tc + 0.5 * cr_td
    if cr_tc_td >= old_tc_td:
        with open(os.path.join(path, 'metrics_config.json'), 'r') as file:
            new = json.load(file)
        if cr_tc_td == old_tc_td:
            TC_TD.append((new, path))
        else:
            old_tc_td = cr_tc_td
            TC_TD = [(new, path)]

print(TC)
print(len(TC))
print('-------------------------------------')
print(TD)
print(len(TD))
print('-------------------------------------')
print(TC_TD)
print(len(TC_TD))

best_results_path = './best_results'
os.makedirs(best_results_path, exist_ok=True)
save_json(TC, os.path.join(best_results_path, 'best_tc.json'))
save_json(TD, os.path.join(best_results_path, 'best_td.json'))
save_json(TC_TD, os.path.join(best_results_path, 'best_tc_td.json'))

In [ ]:
!zip -r best_results/best_results.zip /kaggle/working/results/13